In [1]:
from bs4 import BeautifulSoup as bs
import urllib.request
from nltk import word_tokenize
from nltk.corpus import stopwords
import re
stop = set(stopwords.words('english'))

In [2]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

# Github Extract All Links

In [4]:
def extract_github_page_issues(page_link):
    content = urllib.request.urlopen(page_link)
    data = content.read()
    soup = bs(data,"lxml")
    data = soup.findAll('a', {'class': 'link-gray-dark v-align-middle no-underline h4 js-navigation-open'})
    issues_links = []
    for link in data:
        issues_links.append("https://www.github.com/" + link.get('href'))
    return issues_links

In [5]:
def refine_text(x):
    return [re.sub('[^A-Za-z@.]+', ' ', i.strip()).lower() for i in x]

In [6]:
def extract_issues(link):
    iss = urllib.request.urlopen(link)
    data_1 = iss.read()
    soup_1 = bs(data_1,"lxml")
    data_1 = soup_1.findAll('td',{'class':'d-block comment-body markdown-body js-comment-body'})
    data_1 = data_1 + soup_1.findAll("a",{"class":"title-link"})
    issues = []
    code = []
    code_list = []
    for i in data_1:
        issues.append(i.get_text())
        code_list = code_list + i.findAll("code")
        code_list = code_list + i.findAll("pre")
    for ele in code_list:
        code.append(ele.get_text())
    for id_x in range(len(issues)):
        for ele in code:
            issues[id_x] = issues[id_x].replace(ele," ",1)
    return issues

In [7]:
def extract_repo(REPO):
    try:
        all_links = []
        site = "https://www.github.com/"
        query_page = "/issues?page="
        query_issues = "&q=is%3Aissue"
        for i in range(5):
            print("PageNO : ", i)
            url = site + REPO + query_page + str(i) + query_issues
    #         url = site + REPO + query_page + str(i)
            temp = extract_github_page_issues(url)
            all_links = all_links + temp
            if len(temp) == 0:
                break
        return list(set(all_links))
    except:
        return []

In [8]:
def extract_snyk_issues(link="https://www.snyk.io/vuln/page/"):
    def extract_issues(link):
        print(link)
        iss = urllib.request.urlopen(link)
        data_1 = iss.read()
        soup_1 = bs(data_1,"lxml")
        data_1 = soup_1.findAll('span',{'class':'l-push-left--sm'})
        issues = []
        for ele in data_1:
            issues.append("https://www.snyk.io/" + ele.find("a").get('href'))
        return issues
    end = "?type=golang"
    issues = []
    for i in range(1000):
        temp = extract_issues(link+str(i+1)+end)
        issues = issues + temp 
        if len(temp) == 0:
            break
    
    git_link=[]
    for i in issues:
        print(i)
        iss = urllib.request.urlopen(i)
        data_1 = iss.read()
        soup_1 = bs(data_1,"lxml")
        data_1 = soup_1.findAll('div',{'class':'card card--markdown'})
        for line in data_1:
            for link in line.findAll("a"):
                temp = link.get("href").replace("/"," ")
                if "issues" in temp or "pull" in temp:
                    git_link.append(link.get("href"))
    return list(set(git_link))

In [17]:
links = extract_snyk_issues()

https://www.snyk.io/vuln/page/1?type=golang
https://www.snyk.io/vuln/page/2?type=golang
https://www.snyk.io/vuln/page/3?type=golang
https://www.snyk.io/vuln/page/4?type=golang
https://www.snyk.io//vuln/SNYK-GOLANG-GITHUBCOMGOOGLEGVISORRUNSCBOOTFILTER-72291
https://www.snyk.io//vuln/SNYK-GOLANG-GITHUBCOMMOBYMOBY-72364
https://www.snyk.io//vuln/SNYK-GOLANG-GITHUBCOMGRAFANAGRAFANA-72290
https://www.snyk.io//vuln/SNYK-GOLANG-GITHUBCOMGRAFANAGRAFANAPKGSERVICES-50088
https://www.snyk.io//vuln/SNYK-GOLANG-GITHUBCOMGRAFANAGRAFANAPKGAPI-50087
https://www.snyk.io//vuln/SNYK-GOLANG-GITHUBCOMHASHICORPPACKERBUILDERAMAZONCOMMON-50085
https://www.snyk.io//vuln/SNYK-GOLANG-GITHUBCOMGOOGLEFSCRYPTSECURITY-50084
https://www.snyk.io//vuln/SNYK-GOLANG-GITHUBCOMGOOGLEFSCRYPTPAM-50083
https://www.snyk.io//vuln/SNYK-GOLANG-GITHUBCOMGOGSGOGSROUTESUSER-50081
https://www.snyk.io//vuln/SNYK-GOLANG-GITHUBCOMMINIOMINIOCMD-50080
https://www.snyk.io//vuln/SNYK-GOLANG-GITHUBCOMGOLANGGDDOGOSRC-50079
https://www.snyk.io

In [9]:
def extract_dataset(links):
    doc = []
    for i in links:
        print(i)
        sent = ""
        temp = extract_issues(i)
        for ele in temp:
            sent = sent + ele
        doc.append(sent)
    return doc

In [97]:
links_pos[:10]

['https://www.github.com//F5Networks/k8s-bigip-ctlr/issues/534', 'https://www.github.com//F5Networks/k8s-bigip-ctlr/issues/332', 'https://www.github.com//F5Networks/k8s-bigip-ctlr/issues/95', 'https://www.github.com//F5Networks/k8s-bigip-ctlr/issues/699', 'https://www.github.com//F5Networks/k8s-bigip-ctlr/issues/27', 'https://www.github.com//F5Networks/k8s-bigip-ctlr/issues/143', 'https://www.github.com//F5Networks/k8s-bigip-ctlr/issues/724', 'https://www.github.com//F5Networks/k8s-bigip-ctlr/issues/682', 'https://www.github.com//F5Networks/k8s-bigip-ctlr/issues/142', 'https://www.github.com//F5Networks/k8s-bigip-ctlr/issues/453', 'https://www.github.com//F5Networks/k8s-bigip-ctlr/issues/370', 'https://www.github.com//F5Networks/k8s-bigip-ctlr/issues/84', 'https://www.github.com//F5Networks/k8s-bigip-ctlr/issues/146', 'https://www.github.com//F5Networks/k8s-bigip-ctlr/issues/124', 'https://www.github.com//F5Networks/k8s-bigip-ctlr/issues/147', 'https://www.github.com//F5Networks/k8s-bi

In [ ]:
temp_link = []
for i in git:
    print(i)
    temp_link = temp_link + extract_repo(i)

lodow/argoos
PageNO :  0
heptio/ark
PageNO :  0
PageNO :  1
PageNO :  2
PageNO :  3
PageNO :  4
linki/armor-ingress-controller
PageNO :  0
PageNO :  1
PageNO :  2
pombredanne/atomic-enterprise
PageNO :  0
codem8s/autograph
PageNO :  0
PageNO :  1
PageNO :  2
KarolKraskiewicz/autoscaler
PageNO :  0
giantswarm/aws-operator
PageNO :  0
PageNO :  1
PageNO :  2
PageNO :  3
PageNO :  4
ipedrazas/backpressure
PageNO :  0
ngpitt/blinkt-k8s-controller
PageNO :  0
kubernetes-incubator/bootkube
PageNO :  0
PageNO :  1
PageNO :  2
PageNO :  3
PageNO :  4
michaeltrobinson/cadvisor-integration
PageNO :  0
PageNO :  1
PageNO :  2
PageNO :  3
PageNO :  4
Metaswitch/calico
PageNO :  0
PageNO :  1
PageNO :  2
PageNO :  3
PageNO :  4
ishidawataru/calico-bgp-daemon
PageNO :  0
projectcalico/calico-cni
PageNO :  0
PageNO :  1
PageNO :  2
PageNO :  3
PageNO :  4
ryan-talley/cf-bigip-ctlr
PageNO :  0
kubeflow/chainer-operator
PageNO :  0
PageNO :  1
PageNO :  2
linki/chaoskube
PageNO :  0
PageNO :  1
PageNO 

In [ ]:
te

In [77]:
len(doc)

45

In [10]:
import pickle

# pickle.dump(doc,open("doc.p","wb"))

In [11]:
link = pickle.load( open( "doc.p", "rb" ) )

In [12]:
import random
random.shuffle(link)

In [13]:
selected_pos_links = link[:200]

In [14]:
pos_doc = extract_dataset(selected_pos_links)

https://www.github.com//projectcalico/cni-plugin/issues/299
https://www.github.com//uswitch/kiam/issues/45
https://www.github.com//projectcalico/kube-controllers/issues/35
https://www.github.com//GoogleContainerTools/kaniko/issues/245
https://www.github.com//grafeas/kritis/issues/261
https://www.github.com//linki/chaoskube/issues/15
https://www.github.com//heptio/ark/issues/714
https://www.github.com//GoogleContainerTools/kaniko/issues/365
https://www.github.com//projectcalico/cni-plugin/issues/318
https://www.github.com//kubedb/cli/pull/322
https://www.github.com//uswitch/kiam/issues/52
https://www.github.com//giantswarm/aws-operator/issues/131
https://www.github.com//linki/chaoskube/issues/35
https://www.github.com//kubeflow/chainer-operator/issues/25
https://www.github.com//heptio/ark/issues/808
https://www.github.com//grafeas/kritis/issues/263
https://www.github.com//northwesternmutual/kanali/issues/8
https://www.github.com//GoogleCloudPlatform/k8s-multicluster-ingress/issues/164
h

https://www.github.com//kubeflow/chainer-operator/issues/5
https://www.github.com//projectcalico/cni-plugin/issues/434
https://www.github.com//projectcalico/felix/issues/1745
https://www.github.com//projectcalico/cni-plugin/issues/377
https://www.github.com//uswitch/kiam/issues/149
https://www.github.com//giantswarm/aws-operator/issues/451
https://www.github.com//projectcalico/kube-controllers/issues/62
https://www.github.com//kubernetes-incubator/bootkube/issues/809
https://www.github.com//GoogleCloudPlatform/k8s-stackdriver/issues/170
https://www.github.com//grafeas/kritis/issues/282
https://www.github.com//projectcalico/kube-controllers/issues/87
https://www.github.com//kubernetes-incubator/bootkube/issues/736
https://www.github.com//kubernetes-incubator/bootkube/issues/670
https://www.github.com//projectcalico/felix/issues/1454
https://www.github.com//northwesternmutual/kanali/issues/1
https://www.github.com//krallistic/kafka-operator/issues/41
https://www.github.com//GoogleCloudPl

In [19]:
neg_doc = extract_dataset(links)

https://github.com/opencontainers/runc/pull/708
https://github.com/openshift/origin/pull/8334
https://github.com/robbert229/jwt/issues/12
https://github.com/remind101/empire/pull/1109
https://github.com/kubernetes/kubernetes/issues/43459
https://github.com/bitly/oauth2_proxy/issues/228
https://github.com/golang/go/issues/19767
https://github.com/kubernetes/kubernetes/issues/19479
https://github.com/moby/moby/pull/33182
https://github.com/golang/go/issues/14959
https://github.com/google/fscrypt/issues/77
https://github.com/golang/go/issues/13515
https://github.com/appc/docker2aci/pull/204/commits/54331ec7020e102935c31096f336d31f6400064f
https://github.com/openshift/source-to-image/pull/870
https://github.com/elastic/beats/pull/5457
https://github.com/snapcore/snapd/pull/4194
https://github.com/go-ldap/ldap/pull/126
https://github.com/appc/docker2aci/issues/203
https://github.com/kubernetes/kubernetes/pull/17886
https://github.com/miekg/dns/issues/627
https://github.com/miekg/dns/pull/63

In [16]:
pickle.dump(pos_doc,open("pos_data.p","wb"))

In [20]:
pickle.dump(neg_doc,open("neg_data.p","wb"))

In [53]:
sent = ""
for i in doc:
    sent = sent + i

In [82]:
for i in doc[:10]:
    print(i)


As described here, please change your comparation to default buildin function when hmac is used
With gratitude,
Vetcher

Thanks! I am currently super busy and unable to update this myself. I am open to a PR though!

Closed in #13


When checking the log URL for a pod container we must validate the container name if provided, otherwise the method will return an URL that is actually not valid and results in an error when called, e.g.  .

Labelling this PR as size/L

GCE e2e test build/test passed for commit b2f7d3a.

Build Log
Test Artifacts
Internal Jenkins Results


GCE e2e test build/test passed for commit 44bf2a6.

Build Log
Test Artifacts
Internal Jenkins Results


GCE e2e build/test failed for commit f558dd5.

Build Log
Test Artifacts
Internal Jenkins Results


GCE e2e test build/test passed for commit 325c5a2.

Build Log
Test Artifacts
Internal Jenkins Results


GCE e2e test build/test passed for commit bd90254.

Build Log
Test Artifacts
Internal Jenkins Results


Lgtm, update wi

In [56]:
word = sent.split()

In [57]:
import string
table = str.maketrans('', '', string.punctuation)
stripped = [w.translate(table) for w in word]

In [60]:
import pandas as pd

In [62]:
df = pd.DataFrame({'col':doc})

In [53]:
vectorz = TfidfVectorizer(stop_words = stop, 
                          use_idf=True, ngram_range=(1,3))

In [54]:
X = vectorz.fit_transform(refine_text(d))

In [55]:
X[0]

<1x2310 sparse matrix of type '<class 'numpy.float64'>'
	with 510 stored elements in Compressed Sparse Row format>

# LSA

In [57]:
X.shape

(22, 2310)

In [58]:
lsa = TruncatedSVD(n_components=22,n_iter=100)

In [59]:
lsa.fit(X)

TruncatedSVD(algorithm='randomized', n_components=22, n_iter=100,
       random_state=None, tol=0.0)

In [35]:
lsa.components_[0]

array([1.81834152e-02, 1.81834152e-02, 1.81834152e-02, 1.81834152e-02,
       1.81834152e-02, 1.81834152e-02, 1.81834152e-02, 1.81834152e-02,
       1.81834152e-02, 2.42912981e-02, 2.42912981e-02, 2.42912981e-02,
       6.46039187e-02, 6.46039187e-02, 6.46039187e-02, 9.09170758e-02,
       1.81834152e-02, 1.81834152e-02, 1.81834152e-02, 1.81834152e-02,
       1.81834152e-02, 1.81834152e-02, 1.81834152e-02, 1.81834152e-02,
       1.81834152e-02, 1.81834152e-02, 1.81834152e-02, 1.81834152e-02,
       1.81834152e-02, 1.81834152e-02, 1.81834152e-02, 1.81834152e-02,
       1.81834152e-02, 1.81834152e-02, 1.81834152e-02, 1.81834152e-02,
       1.81834152e-02, 5.45502455e-02, 3.63668303e-02, 3.63668303e-02,
       1.81834152e-02, 1.81834152e-02, 1.81834152e-02, 1.81834152e-02,
       1.81834152e-02, 1.81834152e-02, 1.81834152e-02, 1.81834152e-02,
       1.81834152e-02, 1.81834152e-02, 1.81834152e-02, 3.63668303e-02,
       1.81834152e-02, 1.81834152e-02, 1.81834152e-02, 1.81834152e-02,
      

In [341]:
terms = vectorz.get_feature_names()
for i, comp in enumerate(lsa.components_):
    termInComp = zip(terms,comp)
    sortedterms = sorted(termInComp,key=lambda x:x[1], reverse=True)[:30]
    print("Concept: ", i)
    for term in sortedterms:
        print(term[0])